# Soccer Analytics -  Predicting Soccer Match Results to Improve Chances of Winning Bets 

In [1]:
import numpy as np
import pandas as pd

#### 1. Merge each season individual Team metrics into a single dataset for analysis

In [6]:
def merge_data(name):
    path ='C:\\Documents\\SoccerDataset\\Teamwise_Metrics\\'
    finaldf=pd.DataFrame()
    season=[2014,2015,2016,2017,2018]
    for i in season:
        filename=name+str(i)+'.csv'
        a =filename.find('_')
        df_attackspeed= pd.read_csv(path+filename)
        print("Season:",filename[a+1:a+5],df_attackspeed.shape)
        df_attackspeed['season']=filename[a+1:a+5]
        finaldf=pd.concat([finaldf,df_attackspeed])
    finaldf.to_csv(path+name+'_final.csv')
    
    print(finaldf.shape)

In [7]:
merge_data('attackspeed_')

Season: 2014 (80, 16)
Season: 2015 (80, 16)
Season: 2016 (80, 16)
Season: 2017 (80, 16)
Season: 2018 (80, 16)
(400, 16)


In [8]:
merge_data('situation_')

Season: 2014 (100, 15)
Season: 2015 (100, 15)
Season: 2016 (100, 15)
Season: 2017 (100, 15)
Season: 2018 (100, 15)
(500, 15)


In [10]:
merge_data('gamestate_')

Season: 2014 (100, 17)
Season: 2015 (100, 17)
Season: 2016 (100, 17)
Season: 2017 (100, 17)
Season: 2018 (99, 17)
(499, 17)


In [ ]:
merge_data('formation_')

In [11]:
merge_data('timing_')

Season: 2014 (120, 14)
Season: 2015 (120, 14)
Season: 2016 (120, 14)
Season: 2017 (120, 14)
Season: 2018 (120, 14)
(600, 14)


In [12]:
merge_data('shotzones_')

Season: 2014 (80, 15)
Season: 2015 (80, 15)
Season: 2016 (80, 15)
Season: 2017 (80, 15)
Season: 2018 (78, 15)
(398, 15)


In [ ]:
merge_data('result_')

#### 2. The Scraped scoreboard information was at match level. Here we summarize all the fields present in the scoreboard dataset at season and team level

In [14]:
path ='C:\\Documents\\SoccerDataset\\'

In [15]:
df_team = pd.read_csv(path+'team_scoreboard_seasonwise.csv')

In [16]:
df_team.shape

(3520, 22)

In [17]:
df_team.columns

Index(['id', 'season', 'team', 'h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'ppda',
       'ppda_allowed', 'deep', 'deep_allowed', 'scored', 'missed', 'xpts',
       'result', 'date', 'wins', 'draws', 'loses', 'pts', 'npxGD'],
      dtype='object')

In [18]:
df_team.shape

(3520, 22)

In [19]:
team_final_df =pd.DataFrame(df_team.groupby(['season','team','h_a','id']).sum()).reset_index()
g = pd.DataFrame(df_team.groupby(['season','team','h_a'])['id'].count()).reset_index()
g.rename(columns={"id":"matches"},inplace=True)
final = pd.merge(team_final_df,g,on=['season','team','h_a'])
final['teamid']=final.apply(lambda x : x['team'].replace(' ','_'),axis=1)
final=final[['season', 'id','teamid','team', 'h_a', 'matches',  'wins', 'draws', 'loses','scored', 'missed','deep','deep_allowed', 
       'pts','xG','npxG', 'xGA', 'npxGD',  'npxGA',  'xpts']]

In [20]:
final.columns

Index(['season', 'id', 'teamid', 'team', 'h_a', 'matches', 'wins', 'draws',
       'loses', 'scored', 'missed', 'deep', 'deep_allowed', 'pts', 'xG',
       'npxG', 'xGA', 'npxGD', 'npxGA', 'xpts'],
      dtype='object')

In [21]:
final.to_csv(path+'teamscorebrd_summ.csv')

In [22]:
final.shape

(200, 20)

#### 3. Inserting ID's into Team Metric and Player Performance datasets; will be easier for identification of records and join

In [23]:
ids=pd.DataFrame(final.groupby(['id','team']).size()).reset_index()

In [24]:
ids.rename(columns={0:'count'},inplace=True)

In [42]:
ids

,id,team,count,teamid
0,71,Aston Villa,4,Aston_Villa
1,72,Everton,10,Everton
2,73,Bournemouth,8,Bournemouth
3,74,Southampton,10,Southampton
4,75,Leicester,10,Leicester
5,76,West Bromwich Albion,8,West_Bromwich_Albion
6,77,Sunderland,6,Sunderland
7,78,Crystal Palace,10,Crystal_Palace
8,79,Norwich,2,Norwich
9,80,Chelsea,10,Chelsea


In [26]:
#path ='C:\\Documents\\SoccerDataset\\Player Data\\'
playerdata=pd.read_csv(path+'player_performance_seasonwise.csv')

In [27]:
teamname=[]
teamid=list(ids.id.unique())
teamname=list(ids['team'].unique())
playerteamid=[]
for i in range(0,playerdata.shape[0]):
    for j in range(0,len(teamid)):
        if (playerdata['team_title'][i]==teamname[j]):
            playerteamid.append(teamid[j])
            break
        if (j==28):
            playerteamid.append(' ')
playerdata['team_id']=playerteamid
playerdata = playerdata[['id', 'season', 'player_name','team_id','team_title','games', 'time', 'goals', 'xG',
       'assists', 'xA', 'shots', 'key_passes', 'yellow_cards', 'red_cards',
       'position',  'npg', 'npxG', 'xGChain', 'xGBuildup']]
playerdata.to_csv('C:\\Documents\\SoccerDataset\\EDA_Dataset\\playerdata_summ.csv')

In [28]:
len(teamid)

29

In [29]:
playerteamid=[]
for i in range(0,playerdata.shape[0]):
    for j in range(0,len(teamid)):
        if (playerdata['team_title'][i]==teamname[j]):
            playerteamid.append(teamid[j])
            break
        if (j==28):
            playerteamid.append(' ')

In [30]:
playerdata['team_id']=playerteamid

In [31]:
len(playerteamid)

4227

In [33]:
def get_teamid(df,file):
    teamname=[]
    teamid=list(ids.id.unique())
    teamname=list(ids['teamid'].unique())
    playerteamid=[]
    for i in range(0,df.shape[0]):
        for j in range(0,len(teamid)):
            if (df['team'][i]==teamname[j]):
                playerteamid.append(teamid[j])
                break
            if (j==28):
                playerteamid.append(' ')
    df['team_id']=playerteamid
    df.to_csv('C:\\Documents\\SoccerDataset\\EDA_Dataset\\'+file)

In [34]:
get_teamid(df_result,'result__final.csv')
get_teamid(df_shotzones,'shotzones__final.csv')
get_teamid(df_timing,'timing__final.csv')
get_teamid(df_gamestate,'gamestate__final.csv')
get_teamid(df_formation,'formation__final.csv')
get_teamid(df_situation,'situation__final.csv')
get_teamid(df_attackspeed,'attackspeed__final.csv')

In [35]:
df_result

,Unnamed: 0,Unnamed: 0.1,description,goalsA,goalsF,index,result,season,shotsA,shotsF,team,xGA,xGA/Sh,xGD,xGF,xGF/Sh,team_id
0,0,0,Aston Villa xG stats for the 2014/2015 season,0.0,0,BlockedShot,BlockedShot,2014,142.0,105,Aston_Villa,8.001862,0.056351,-2.310036,5.691826,0.054208,71
1,1,1,Aston Villa xG stats for the 2014/2015 season,57.0,31,Goal,Goal,2014,57.0,31,Aston_Villa,18.774121,0.329371,-12.696147,6.077973,0.196064,71
2,2,2,Aston Villa xG stats for the 2014/2015 season,0.0,0,MissedShots,MissedShots,2014,159.0,183,Aston_Villa,10.600322,0.066669,1.766608,12.366930,0.067579,71
3,3,3,Aston Villa xG stats for the 2014/2015 season,0.0,0,SavedShot,SavedShot,2014,112.0,93,Aston_Villa,12.722259,0.113592,-3.945346,8.776913,0.094375,71
4,4,4,Aston Villa xG stats for the 2014/2015 season,0.0,0,ShotOnPost,ShotOnPost,2014,11.0,5,Aston_Villa,1.466639,0.133331,-1.186053,0.280586,0.056117,71
5,5,0,Everton xG stats for the 2014/2015 season,0.0,0,BlockedShot,BlockedShot,2014,176.0,141,Everton,9.068478,0.051525,-0.885438,8.183040,0.058036,72
6,6,1,Everton xG stats for the 2014/2015 season,50.0,48,Goal,Goal,2014,50.0,48,Everton,17.777890,0.355558,-0.969111,16.808780,0.350183,72
7,7,2,Everton xG stats for the 2014/2015 season,0.0,0,MissedShots,MissedShots,2014,193.0,173,Everton,14.668382,0.076002,-3.528221,11.140161,0.064394,72
8,8,3,Everton xG stats for the 2014/2015 season,0.0,0,SavedShot,SavedShot,2014,80.0,116,Everton,6.212380,0.077655,3.079921,9.292301,0.080106,72
9,9,4,Everton xG stats for the 2014/2015 season,0.0,0,ShotOnPost,ShotOnPost,2014,6.0,7,Everton,0.838604,0.139767,0.828472,1.667077,0.238154,72


In [36]:
final

,season,id,teamid,team,h_a,matches,wins,draws,loses,scored,missed,deep,deep_allowed,pts,xG,npxG,xGA,npxGD,npxGA,xpts
0,2014,83,Arsenal,Arsenal,a,19,10,4,5,30,22,166,93,34,29.267701,26.223024,20.091949,6.892248,19.330776,32.4731
1,2014,83,Arsenal,Arsenal,h,19,12,5,2,41,14,232,78,41,40.534890,38.251380,15.626876,24.146804,14.104576,42.7009
2,2014,71,Aston_Villa,Aston Villa,a,19,5,2,12,13,32,85,113,17,14.202126,14.202126,27.909755,-12.185289,26.387415,17.2619
3,2014,71,Aston_Villa,Aston Villa,h,19,5,6,8,18,25,99,124,21,18.900545,18.139375,21.796501,-2.134789,20.274164,23.6298
4,2014,92,Burnley,Burnley,a,19,3,5,11,14,32,107,149,14,18.796987,18.035818,33.069350,-12.750024,30.785842,17.0794
5,2014,92,Burnley,Burnley,h,19,4,7,8,14,21,111,109,19,20.575662,19.053322,24.819506,-4.243849,23.297171,22.7165
6,2014,80,Chelsea,Chelsea,a,19,11,5,3,37,23,187,105,38,28.911835,28.911835,19.090902,11.343272,17.568563,32.4742
7,2014,80,Chelsea,Chelsea,h,19,15,4,0,36,9,220,66,49,39.731485,35.828567,12.433442,24.156295,11.672272,42.8430
8,2014,78,Crystal_Palace,Crystal Palace,a,19,7,6,6,26,24,76,155,27,20.274114,16.331510,25.677562,-6.965500,23.297010,21.1489
9,2014,78,Crystal_Palace,Crystal Palace,h,19,6,3,10,21,27,104,103,21,24.482569,23.721399,19.806803,6.198105,17.523294,29.4081


#### 4. Creation of team dimension table for EDA and dashboard creation in Qlik

In [37]:
report=pd.DataFrame(final.groupby(['season','id','team']).size()).reset_index()

In [38]:
report.drop(columns=[0],inplace=True)

In [39]:
report['identifier']=report.season.map(str)+report.id.map(str)

In [40]:
report

,season,id,team,identifier
0,2014,71,Aston Villa,201471
1,2014,72,Everton,201472
2,2014,74,Southampton,201474
3,2014,75,Leicester,201475
4,2014,76,West Bromwich Albion,201476
5,2014,77,Sunderland,201477
6,2014,78,Crystal Palace,201478
7,2014,80,Chelsea,201480
8,2014,81,West Ham,201481
9,2014,82,Tottenham,201482


In [41]:
report.to_csv('C:\\Documents\\SoccerDataset\\EDA_Dataset\\teamdimension.csv')

#### 5. Appending TeamID's in the match stats dataset

In [43]:
ids

,id,team,count,teamid
0,71,Aston Villa,4,Aston_Villa
1,72,Everton,10,Everton
2,73,Bournemouth,8,Bournemouth
3,74,Southampton,10,Southampton
4,75,Leicester,10,Leicester
5,76,West Bromwich Albion,8,West_Bromwich_Albion
6,77,Sunderland,6,Sunderland
7,78,Crystal Palace,10,Crystal_Palace
8,79,Norwich,2,Norwich
9,80,Chelsea,10,Chelsea


In [44]:
path = 'C:\\Documents\\SoccerDataset\\EDA_Dataset\\'
match_df=pd.read_csv(path+'team_match_stats.csv')

In [47]:
match_df.drop(columns={'Unnamed: 0','Unnamed: 0.1'},inplace=True)

In [56]:
teamname=[]
teamid=list(ids.id.unique())
teamname=list(ids['team'].unique())
matchhometeamid=[]
matchawayteamid=[]
for i in range(0,match_df.shape[0]):
    for j in range(0,len(teamid)):
        if (match_df['Hteam'][i]==teamname[j]):
            matchhometeamid.append(teamid[j])
        if (match_df['Ateam'][i]==teamname[j]):
            matchawayteamid.append(teamid[j])
        if (j==28):
            playerteamid.append(' ')
match_df['Hteam_id']=matchhometeamid
match_df['Ateam_id']=matchawayteamid

In [57]:
match_df.columns

Index(['Hteam', 'Hchance', 'Hgoals', 'HxG', 'Hshots', 'Hshotstrgt', 'Hdeep',
       'Hppda', 'Hxpts', 'Ateam', 'Achance', 'Agoals', 'AxG', 'Ashots',
       'Ashotstrgt', 'Adeep', 'Appda', 'Axpts', 'Dchance', 'FTR', 'Season',
       'League', 'Description', 'MatchDate', 'Hteam_id', 'Ateam_id'],
      dtype='object')

In [58]:
matchdata = match_df[['Season',
       'League', 'Description', 'MatchDate', 'Hteam_id', 'Hteam', 'Hchance', 'Hgoals', 'HxG', 'Hshots', 'Hshotstrgt', 'Hdeep',
       'Hppda', 'Hxpts', 'Ateam','Ateam_id', 'Achance', 'Agoals', 'AxG', 'Ashots',
       'Ashotstrgt', 'Adeep', 'Appda', 'Axpts', 'Dchance', 'FTR']]
matchdata.to_csv('C:\\Documents\\SoccerDataset\\EDA_Dataset\\team_match_stats_v2.csv')

In [59]:
matchdata.head()

,Season,League,Description,MatchDate,Hteam_id,Hteam,Hchance,Hgoals,HxG,Hshots,...,Achance,Agoals,AxG,Ashots,Ashotstrgt,Adeep,Appda,Axpts,Dchance,FTR
0,2014/2015,EPL,West Bromwich Albion 3 - 0 Chelsea (May 18 2015),May 18 2015),76,West Bromwich Albion,40%,3,1.23,9,...,28%,0,1.08,12,3,7,12.32,1.15,32%,West Bromwich Albion
1,2014/2015,EPL,Arsenal 0 - 0 Sunderland (May 20 2015),May 20 2015),83,Arsenal,39%,0,1.69,28,...,35%,0,1.50,7,3,6,27.82,1.30,26%,Draw
2,2014/2015,EPL,Aston Villa 0 - 1 Burnley (May 24 2015),May 24 2015),71,Aston Villa,28%,0,0.65,16,...,31%,1,0.66,6,3,6,19.64,1.33,41%,Burnley
3,2014/2015,EPL,Chelsea 3 - 1 Sunderland (May 24 2015),May 24 2015),80,Chelsea,66%,3,2.53,24,...,NaN,1,1.39,17,9,1,19.93,0.62,20%,Chelsea
4,2014/2015,EPL,Crystal Palace 1 - 0 Swansea (May 24 2015),May 24 2015),78,Crystal Palace,68%,1,1.08,13,...,NaN,0,0.12,5,2,1,9.17,0.38,29%,Crystal Palace


In [63]:
matchdata.isna().any()

Season         False
League         False
Description    False
MatchDate      False
Hteam_id       False
Hteam          False
Hchance         True
Hgoals         False
HxG            False
Hshots         False
Hshotstrgt     False
Hdeep          False
Hppda          False
Hxpts          False
Ateam          False
Ateam_id       False
Achance         True
Agoals         False
AxG            False
Ashots         False
Ashotstrgt     False
Adeep          False
Appda          False
Axpts          False
Dchance         True
FTR            False
dtype: bool